In [ ]:
import pandas as pd
import re

# Gantilah 'nama_file.csv' dengan nama file yang sesuai dengan data Twitter Anda
df = pd.read_csv('twtrr1.csv')

# Hapus semua kolom kecuali 'Contentp' dan 'Label'
new = ['Contentp', 'Label']
df = df[new]

# Tampilkan DataFrame yang telah diubah
df.head()

,Contentp,Label
0,polda gelar pemilu damai bersama perwakilan pa...,infoSumbar
1,danden gegana satbrimob polda kompol mulyadi p...,Geganasumbar
2,keluarga besar detasemen gegana satuan brimob ...,Geganasumbar
3,sebanyak ditangkap kepolisian daerah polda pem...,suaradotcom
4,aksi berlangsung damai wakil bupati pasaman be...,bersihkan_indo


In [ ]:
import re

# Buat dictionary untuk mencocokkan kata kunci dengan provinsi
keyword_provinsi = {
    'Aceh': r'aceh',
    'Sumatera Utara': r'sumatera[ ]*utara|sumut',
    'Sumatera Barat': r'sumatera[ ]*barat|sumbar',
    'Riau': r'riau',
    'Kepulauan Riau': r'kepulauan[ ]*riau',
    'Jambi': r'jambi',
    'Bengkulu': r'bengkulu',
    'Sumatera Selatan': r'sumatera[ ]*selatan|sumsel',
    'Lampung': r'lampung',
    'Bangka Belitung': r'bangka[ ]*belitung',
    'DKI Jakarta': r'dki[ ]*jakarta|jakarta',
    'Jawa Barat': r'jawa[ ]*barat|jabar',
    'Jawa Tengah': r'jawa[ ]*tengah|jateng',
    'DI Yogyakarta': r'di[ ]*yogyakarta|yogyakarta|jogja',
    'Jawa Timur': r'jawa[ ]*timur|jatim',
    'Banten': r'banten',
    'Bali': r'bali',
    'Nusa Tenggara Barat': r'nusa[ ]*tenggara[ ]*barat|ntb',
    'Nusa Tenggara Timur': r'nusa[ ]*tenggara[ ]*timur|ntt',
    'Kalimantan Barat': r'kalimantan[ ]*barat|kalbar',
    'Kalimantan Tengah': r'kalimantan[ ]*tengah|kalteng',
    'Kalimantan Selatan': r'kalimantan[ ]*selatan|kalsel',
    'Kalimantan Timur': r'kalimantan[ ]*timur|kaltim',
    'Kalimantan Utara': r'kalimantan[ ]*utara|kalut',
    'Gorontalo': r'gorontalo',
    'Sulawesi Utara': r'sulawesi[ ]*utara|sulut',
    'Sulawesi Tengah': r'sulawesi[ ]*tengah|sulteng',
    'Sulawesi Selatan': r'sulawesi[ ]*selatan|sulsel',
    'Sulawesi Tenggara': r'sulawesi[ ]*tenggara|sulteng',
    'Sulawesi Barat': r'sulawesi[ ]*barat|sulbar',
    'Maluku': r'maluku',
    'Maluku Utara': r'maluku[ ]*utara|malut',
    'Papua Barat': r'papua[ ]*barat',
    'Papua': r'papua',
}

# Dictionary kota dengan huruf depan kota dalam huruf kecil
kota_provinsi = {
    'Aceh': r'banda Aceh', r'lhokseumawe', r'langsa',
    'Sumatera Utara': r'medan', r'binjai', r'pematangsiantar',
    'Sumatera Barat': r'padang', r'bukittinggi', r'pariaman',
    'Riau': r'pekanbaru', r'dumai', r'tanjungpinang',
    'Kepulauan Riau': r'tanjungpinang', r'batam', r'tanjung Balai Karimun',
    'Jambi': r'muara Bungo', r'kerinci',
    'Bengkulu': r'curup', r'manna',
    'Sumatera Selatan': r'palembang', r'prabumulih', r'lubuklinggau',
    'Lampung': r'bandar Lampung', r'metro', r'liwa',
    'Bangka Belitung': r'pangkal Pinang', r'sungailiat', r'belinyu',
    'DKI Jakarta': r'jakarta Pusat', r'jakarta Selatan', r'jakarta Barat',
    'Jawa Barat': r'bandung', r'bogor', r'depok', r'bekasi',
    'Jawa Tengah': r'semarang', r'surakarta', r'tegal',
    'DI Yogyakarta': r'yogyakarta', r'sleman', r'bantul',
    'Jawa Timur': r'surabaya', r'malang', r'sidoarjo',
    'Banten': r'serang', r'tangerang', r'cilegon',
    'Bali': r'denpasar', r'badung', r'ubud',
    'Nusa Tenggara Barat': r'mataram', r'bima', r'sumbawa Besar',
    'Nusa Tenggara Timur': r'kupang', r'ende', r'maumere',
    'Kalimantan Barat': r'pontianak', r'singkawang', r'ketapang',
    'Kalimantan Tengah': r'palangkaraya', r'sampit', r'pangkalan Bun',
    'Kalimantan Selatan': r'balikpapan', r'banjarmasin', r'palangkaraya',
    'Kalimantan Timur': r'samarinda', r'bontang', r'tarakan',
    'Kalimantan Utara': r'tanjung Selor', r'tarakan', r'tideng Pale',
    'Gorontalo': r'kota Baru', r'limboto',
    'Sulawesi Utara': r'manado', r'bitung', r'tondano',
    'Sulawesi Tengah': r'palu', r'donggala', r'toli-Toli',
    'Sulawesi Selatan': r'makassar', r'gowa', r'palopo',
    'Sulawesi Tenggara': r'kendari', r'unaaha', r'baubau',
    'Sulawesi Barat': r'mamuju', r'majene', r'polewali Mandar',
    'Maluku': r'ambon', r'tual', r'saumlaki',
    'Maluku Utara': r'ternate', r'tidore', r'sofifi',
    'Papua Barat': r'manokwari', r'sorong', r'fakfak',
    'Papua': r'jayapura', r'biak', r'merauke',
}

# Fungsi untuk mencocokkan kata kunci dengan provinsi
def cari_provinsi(text):
    for provinsi, pattern in keyword_provinsi.items():
        if re.search(pattern, text, re.IGNORECASE):
            return provinsi
    for provinsi, kota_list in kota_provinsi.items():
        for kota_pattern in kota_list:
            if re.search(kota_pattern, text, re.IGNORECASE):
                return provinsi
    return 'Lainnya'  # Jika tidak ada provinsi atau kota yang cocok, kategorikan sebagai 'Lainnya'

# Buat kolom baru 'Provinsi' berdasarkan kata kunci
df['Provinsi'] = df['Contentp'].apply(cari_provinsi)

# Tampilkan DataFrame yang telah diubah
df.head()

,Contentp,Label,Provinsi
0,polda gelar pemilu damai bersama perwakilan pa...,infoSumbar,Sumatera Barat
18,jasa raharja cabang mengucapkan selamat atas p...,jr_sumbar9,Sumatera Barat
23,rangka bhayangkara dokkes polda menggelar jala...,jr_sumbar9,Sumatera Barat
26,polri amankan korban tppo polri melalui polda ...,Bimo41521318,Nusa Tenggara Barat
44,diskominfo banda aceh pemakaman wadansat brimo...,sinergi_bna,Aceh


In [ ]:
# Hapus isi kolom 'Contentp' dan 'Label' jika nilai 'Provinsi' adalah 'Lainnya'
df.loc[df['Provinsi'] == 'Lainnya', ['Contentp', 'Label']] = ''

# Hapus nilai 'Lainnya' dari kolom 'Provinsi'
df = df[df['Provinsi'] != 'Lainnya']

# Tampilkan DataFrame yang telah diubah
df.head()

,Contentp,Label,Provinsi
0,polda gelar pemilu damai bersama perwakilan pa...,infoSumbar,Sumatera Barat
18,jasa raharja cabang mengucapkan selamat atas p...,jr_sumbar9,Sumatera Barat
23,rangka bhayangkara dokkes polda menggelar jala...,jr_sumbar9,Sumatera Barat
26,polri amankan korban tppo polri melalui polda ...,Bimo41521318,Nusa Tenggara Barat
44,diskominfo banda aceh pemakaman wadansat brimo...,sinergi_bna,Aceh


In [ ]:
df.to_csv('twtnew6.csv', index=False)